# Landsat 8 Classification Example

This example will walk you through an example of using DELTA to train a simple example model. You can use what you learn here to use DELTA on your own datasets and with your own model architectures.
In this example you will:
- Download a dataset of images and labels
- Train a simple model using example configuration files
- Examine results of the trained model
- Make some changes to the configuration files and train a modified model
- Examine the results of the modified model

## Downloading and Extracting Dataset

The dataset includes satellite images along with classification labels for different types of land cover (water, cloud, snow, etc.).


In [1]:
!echo "Downloading dataset."
!curl -O https://landsat.usgs.gov/cloud-validation/sparcs/l8cloudmasks.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1483M  100 1483M    0     0  2746k      0  0:09:13  0:09:13 --:--:-- 5555k15  0:00:18  0:08:57 1938k1  0:09:05 2236k2681k      0  0:09:26  0:00:26  0:09:00 2682k9 3187k3 3289k6k      0  0:08:37  0:01:04  0:07:33 1407k2836k      0  0:08:55  0:01:08  0:07:47 1279k0     0  2477k      0  0:10:13  0:01:42  0:08:31 3007k 0  0:09:19  0:02:15  0:07:04 4545k 2732k      0  0:09:15  0:02:25  0:06:50 2926k1  0:02:59  0:06:22 1946k0:09:23  0:03:03  0:06:20 2334k 0  0:09:34  0:03:17  0:06:17  599k  0  0:09:39  0:03:20  0:06:19 1247k   0  2583k      0  0:09:48  0:03:35  0:06:13 1979k09:49  0:03:50  0:05:59 2954k     0  0:09:51  0:04:22  0:05:29 3312k 2587k      0  0:09:47  0:04:26  0:05:21 3763k      0  0:09:46  0:04:37  0:05:09 2739k9:37  0:04:49  0:04:48 3964k:05:17  0:04:21 2398k    0  0:09:40  0:05:26  0:04:14 2520k2619k      0  0:09:40 

Here we're extracting the dataset and organizing the images into folders.
We are:
- setting aside two images in a folder called "validate" to test our model later
- moving the satellite images into a folder called "train"
- moving the classification labels into a folder called "labels"

In [2]:
!echo "Extracting dataset."
!unzip -q l8cloudmasks.zip
!mkdir validate
!mv sending/LC82290562014157LGN00_24_data.tif sending/LC82210662014229LGN00_18_data.tif validate/
!mkdir train
!mv sending/*_data.tif train/
!mkdir labels
!mv sending/*_mask.png labels/

Extracting dataset.
mkdir: validate: File exists
mkdir: train: File exists
mkdir: labels: File exists


In [3]:
!ls

example_screenshots            l8_cloud_train_parameters.yaml
l8_cloud.sh                    l8cloudmasks.zip
l8_cloud_dataset.yaml          labels
l8_cloud_dataset_water.yaml    sending
l8_cloud_example.ipynb         train
l8_cloud_train_network.yaml    validate


In [4]:
!ls train/ | head

LC80010812013365LGN00_18_data.tif
LC80020622013244LGN00_32_data.tif
LC80050152014172LGN00_12_data.tif
LC80050562014076LGN00_33_data.tif
LC80150242014146LGN00_23_data.tif
LC80190352014078LGN01_26_data.tif
LC80200462014213LGN00_11_data.tif
LC80250402013245LGN00_45_data.tif
LC80250482014072LGN00_18_data.tif
LC80310432013207LGN00_11_data.tif


### Example Satellite Image
<img src='example_screenshots/Screen Shot 2021-07-13 at 6.57.11 PM.png'>

In [5]:
!ls labels | head

LC80010812013365LGN00_18_mask.png
LC80020622013244LGN00_32_mask.png
LC80050152014172LGN00_12_mask.png
LC80050562014076LGN00_33_mask.png
LC80150242014146LGN00_23_mask.png
LC80190352014078LGN01_26_mask.png
LC80200462014213LGN00_11_mask.png
LC80250402013245LGN00_45_mask.png
LC80250482014072LGN00_18_mask.png
LC80310432013207LGN00_11_mask.png


### Example Label Image

The different colors represent different land cover classifications.

<img src='labels/LC80010812013365LGN00_18_mask.png'>

## Training a Model

We'll use the following configuration files which specify a dataset to use (l8_cloud_dataset.yaml), a network architecture to train (l8_cloud_train_network.yaml), and the parameters to train with (l8_cloud_train_parameters.yaml).

### Configuration YAMLs

<a href='./l8_cloud_dataset.yaml' > l8_cloud_dataset.yaml* </a><br />
<a href='./l8_cloud_train_network.yaml' > l8_cloud_train_network.yaml* </a><br />
<a href='./l8_cloud_train_parameters.yaml' > l8_cloud_train_parameters.yaml* </a>

<a href='https://github.com/nasa/delta/blob/master/delta/config/README.md' > Detailed Config Documentation </a>

_*If you're viewing this notebook on Github, these links will be broken. You can find these .yaml files in the same folder this notebook is in or view a more native version of this notebook by pasting the current URL at [https://nbviewer.org](https://nbviewer.org)._

In [6]:
!delta train --config l8_cloud_dataset.yaml --config l8_cloud_train_network.yaml --config l8_cloud_train_parameters.yaml l8_clouds.SavedModel 2>/dev/null

Epoch 1/10
31/31 [==============================] - 304s 10s/step - loss: 1.1059 - sparse_categorical_accuracy: 0.6334

## Examine Model Results

The previous step produced a trained model. Now we can use the model to classify the images we set aside in the "validate" folder.

In [12]:
!delta classify --config l8_cloud_dataset.yaml --image-dir ./validate --outdir ./model_output --overlap 32 l8_clouds.SavedModel 2>/dev/null

LC82290562014157LGN00_24_data : |████████████████████████████████████████████████████████████████████████████████| 9 / 9
Image: ./validate/LC82290562014157LGN00_24_data.tif
Shadow              --- Precision:  58.43%    Recall:  39.25%        Frequency:  13.64%
Shadow over Water   --- Precision:   0.00%    Recall:   0.00%        Frequency:   3.84%
Water               --- Precision:  77.45%    Recall:  94.50%        Frequency:  24.57%
Snow                --- Precision:   0.00%    Recall:   0.00%        Frequency:   0.00%
Land                --- Precision:  75.19%    Recall:  96.38%        Frequency:  29.91%
Cloud               --- Precision:  96.24%    Recall:  77.30%        Frequency:  28.05%
Flooded             --- Precision:   0.00%    Recall:   0.00%        Frequency:   0.00%
 79.07% Accuracy

LC82210662014229LGN00_18_data : |████████████████████████████████████████████████████████████████████████████████| 9 / 9
Image: ./validate/LC82210662014229LGN00_18_data.tif
Shadow              

### Validation Image
The validation image we fed into the model.

<img src='example_screenshots/Screen Shot 2021-07-14 at 1.35.36 PM.png'>

### Validation Model Output
The output from the model classifying the land cover in the above image.

<img src='example_screenshots/Screen Shot 2021-07-14 at 1.36.41 PM.png'>

## Train Modified Model

The previous model we trained classified all the land cover types in the landsat 8 dataset (water, snow, clouds, etc.). Now we're going to make a simple modification to the dataset configuration file and train a model that just classifies water in the satellite image.

All we have to do is add one of DELTA's built in preprocessing functions. It will map all the of the classes except water to one class and set water as the other class.

<a href='./l8_cloud_dataset_water.yaml' > l8_cloud_dataset_water.yaml* </a><br />

Excerpt from l8_cloud_dataset_water.yaml :
```yaml
# ______________
# this mapping section tells DELTA to set all
# the classes EXCEPT water to 0 and the water
# claass to 2
# ______________
    preprocess:
      - substitute:
          mapping:
            - 0
            - 0
            - 2
            - 0
            - 0
            - 0
            - 0
  classes:
    - 0:
        name: Shadow
        color: 0x000000
    - 1:
        name: Shadow over Water
        color: 0x000080
    - 2:
        name: Water
        color: 0x0000FF
    - 3:
        name: Snow
        color: 0x00FFFF
    - 4:
        name: Land
        color: 0x808080
    - 5:
        name: Cloud
        color: 0xFFFFFF
    - 6:
        name: Flooded
        color: 0x808000
```

_*If you're viewing this notebook on Github, these links will be broken. You can find these .yaml files in the same folder this notebook is in or view a more native version of this notebook by pasting the current URL at [https://nbviewer.org](https://nbviewer.org)._

In [8]:
!delta train --config l8_cloud_dataset_water.yaml --config l8_cloud_train_network.yaml --config l8_cloud_train_parameters.yaml l8_clouds_water.SavedModel 2>/dev/null

Epoch 1/10
31/31 [==============================] - 227s 7s/step - loss: 0.7085 - sparse_categorical_accuracy: 0.8178

## Examine Model Results

Now we can examine the new model by classifying the images we set aside in the "validate" folder.

In [13]:
!delta classify --config l8_cloud_dataset_water.yaml --image-dir ./validate --outdir ./model_output_water --overlap 32 l8_clouds_water.SavedModel 2>/dev/null

LC82290562014157LGN00_24_data : |████████████████████████████████████████████████████████████████████████████████| 9 / 9
Image: ./validate/LC82290562014157LGN00_24_data.tif
Shadow              --- Precision:  98.83%    Recall:  93.78%        Frequency:  75.43%
Shadow over Water   --- Precision:   0.00%    Recall:   0.00%        Frequency:   0.00%
Water               --- Precision:  83.48%    Recall:  96.59%        Frequency:  24.57%
Snow                --- Precision:   0.00%    Recall:   0.00%        Frequency:   0.00%
Land                --- Precision:   0.00%    Recall:   0.00%        Frequency:   0.00%
Cloud               --- Precision:   0.00%    Recall:   0.00%        Frequency:   0.00%
Flooded             --- Precision:   0.00%    Recall:   0.00%        Frequency:   0.00%
 94.47% Accuracy

LC82210662014229LGN00_18_data : |████████████████████████████████████████████████████████████████████████████████| 9 / 9
Image: ./validate/LC82210662014229LGN00_18_data.tif
Shadow              

### Validation Image
The validation image we fed into the model.

<img src='example_screenshots/Screen Shot 2021-07-14 at 1.35.36 PM.png'>

### Validation Model Output
The output from the model classifying water coverage in the above image.

<img src='example_screenshots/Screen Shot 2021-07-14 at 2.17.03 PM.png'>

